In [62]:

import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from bson.son import SON
from bson.code import Code
import json
import pandas as pd
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
# These are packages need for natural language processing:
import nltk
from __future__ import division, unicode_literals 
from textblob import TextBlob as tb
import math

In [6]:
client = MongoClient()
#creating mbf database (mbf = "meta beauty file")
mbf = client.dsbc.mbf
# mbf.remove()


In [ ]:
with open('meta_Beauty.json') as meta_beauty_file:
    for line in meta_beauty_file:
        meta_f = json.loads(line)  
        #only work when using json.loads() not json.load() 
        mbf.insert(meta_f)
        
#can't do json.load(meta_beautfy_file) since meta_beauty_file has the form "{...}{...}{...}" --> not json
#json.load(line) does not work since line is  string

In [59]:
# "categories": [["Beauty", "Skin Care",...]] --> need to $unwind
# use "categories.0" get the innner list.

#Skin_care = mbf.find({"categories.0": ["Beauty"]})

pipeline  = [{"$unwind":"$categories"},
             {"$match":{"categories":{"$all":["Face","Creams & Moisturizers"]}}},
             {"$project":{"_id":0,"asin":1,"title":1,"price":1}},
             {"$out": "product_info"}]
# "$out" write the result into new collection
# "$all" find all list that contains everything in queries
mbf.aggregate(pipeline)

In [12]:
product_info = client.dsbc.product_info

In [13]:
# name = product_info.find({}, {"_id":0,"asin":1})
# id_list  = {}
# for id_num in name:
#     id_list.append(id_num["asin"])

In [15]:
name = product_info.find({}, {"_id":0,"asin":1,"title":1,"price":1})
id_list  = {}
for id_num in name:
    try:
        id_list[id_num["asin"]]=[id_num["title"],id_num["price"]]
    except:
        pass


In [82]:
#id_list.keys()

In [17]:
#Creating database for reviews beauty file
rbf = client.dsbc.rbf

In [10]:

with open('reviews_Beauty.json') as reviews_beauty_file:
    for line in reviews_beauty_file:
        review = json.loads(line)  
        rbf.insert(review)
        

In [18]:
#new database for selective items
ndb = client.dsbc.ndb

In [60]:
beauty_item_info = {id_list.keys()[0] : id_list.values()[0], 
                    "Reviews" : [], "rating":{"Five_star":0, 
                                              "Four_star":0,
                                              "Three_star":0,
                                              "Two_star":0,
                                              "One_star":0}}
match_list = rbf.find({"asin":{"$in": [id_list.keys()[0]]}})

for i in match_list:
    beauty_item_info["Reviews"].append(tb(i['reviewText']))
    if i['overall'] == 5.0:
        beauty_item_info["rating"]["Five_star"] += 1
    elif i['overall'] == 4.0:
        beauty_item_info["rating"]["Four_star"] += 1
    elif i['overall'] == 3.0:
        beauty_item_info["rating"]["Three_star"] += 1
    elif i['overall'] == 2.0:
        beauty_item_info["rating"]["Two_star"] += 1
    else:
        beauty_item_info["rating"]["One_star"] += 1
        
        

#TF-IDF method

tf(word, blob) computes "term frequency" which is the number of times a word appears in a document blob, normalized by dividing by the total number of words in blob. We use TextBlob for breaking up the text into words and getting the word counts.

n_containing(word, bloblist) returns the number of documents containing word. A generator expression is passed to the sum() function.

idf(word, bloblist) computes "inverse document frequency" which measures how common a word is among all documents in bloblist. The more common a word is, the lower its idf. We take the ratio of the total number of documents to the number of documents containing word, then take the log of that. Add 1 to the divisor to prevent division by zero.
tfidf(word, blob, bloblist) computes the TF-IDF score. It is simply the product of tf and idf.

In [63]:
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [66]:
for i, blob in enumerate(beauty_item_info['Reviews']):
    print i, blob.sentiment

0 Sentiment(polarity=0.3722222222222222, subjectivity=0.6432098765432098)
1 Sentiment(polarity=0.2, subjectivity=0.6)


In [65]:
beauty_item_info['Reviews']

[TextBlob("I just started using this, so I'm not sure yet of the long-term effects, but overall it's a great moisturizer.  I'm fairly picky about moisturizers, since I tend to be really oily (my whole face, too...lucky me).  This one seems like it might be a winner.  It absorbed nicely and my skin did look and feel great after using it. It did lose one star, though, because of the price- it's a bit steep."),
 TextBlob("I bought it for a collagen moisturizer and immediately found it cleared my breakouts. I still use an under eye cream since I am 56 but this cream is miraculous. It soaks in so fast you can't believe it and it not greasy or shiny for someone with oily skin.")]

In [69]:
for i, blob in enumerate(beauty_item_info['Reviews']):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, beauty_item_info['Reviews']) for word in blob.words}
    print scores
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1
{u'and': -0.010264939445776314, u'winner': 0.0, u'just': 0.0, u'be': -0.010264939445776314, u'feel': 0.0, u'look': 0.0, u'it': -0.030794818337328944, u'one': -0.010264939445776314, u'because': 0.0, u'skin': -0.005132469722888157, u'star': 0.0, u'tend': 0.0, u'moisturizer': -0.005132469722888157, u'really': 0.0, u'to': 0.0, u'since': -0.005132469722888157, u'seems': 0.0, u'yet': 0.0, u'started': 0.0, u'absorbed': 0.0, u'whole': 0.0, u'bit': 0.0, u'This': 0.0, u'might': 0.0, u'oily': -0.005132469722888157, u'too': 0.0, u'fairly': 0.0, u'sure': 0.0, u"'s": 0.0, u'steep': 0.0, u'though': 0.0, u'I': -0.02052987889155263, u'moisturizers': 0.0, u'price': 0.0, u'after': 0.0, u'overall': 0.0, u'about': 0.0, u'but': -0.005132469722888157, u'lucky': 0.0, u'me': -0.005132469722888157, u'effects': 0.0, u'not': -0.005132469722888157, u'using': 0.0, u"'m": 0.0, u'like': 0.0, u'a': -0.015397409168664472, u'great': 0.0, u'picky': 0.0, u'this': -0.010264939445776314, u'of': 0.0, 

###Based on the above analysis, we can see that it's not very helpful to look at TF-IDF

# Document Summarization

In [71]:
beauty_item_info = {id_list.keys()[0] : id_list.values()[0], 
                    "Reviews" : [], "rating":{"Five_star":0, 
                                              "Four_star":0,
                                              "Three_star":0,
                                              "Two_star":0,
                                              "One_star":0}}
match_list = rbf.find({"asin":{"$in": [id_list.keys()[0]]}})

for i in match_list:
    beauty_item_info["Reviews"].append(i['reviewText'])
    if i['overall'] == 5.0:
        beauty_item_info["rating"]["Five_star"] += 1
    elif i['overall'] == 4.0:
        beauty_item_info["rating"]["Four_star"] += 1
    elif i['overall'] == 3.0:
        beauty_item_info["rating"]["Three_star"] += 1
    elif i['overall'] == 2.0:
        beauty_item_info["rating"]["Two_star"] += 1
    else:
        beauty_item_info["rating"]["One_star"] += 1

In [72]:
text = ''
for i in beauty_item_info['Reviews']:
    text += i

In [80]:
N = 100 #number of words to consider
Cluster_threshold = 5 #distance between two words to consider
Top_sentences = 5 #number of sentences to return for a "top n" summary

In [81]:
def _score_sentences(sentences, important_words,Cluster_threshold = 5):
    scores = []
    sentence_idx = -1
    
    #word_tokenize(s) split a sentence and find all the words and punctuations
    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:
        sentence_idx += 1
        word_idx =[]
        
        # This function check whether any important words occur in the sentence
        # if  there is, record the location of the important word in the sentence
        for w in important_words:
            try:
                word_idx.append(s.index(w))
            except ValueError, e:
                pass
        word_idx.sort()
        
        #If there is no important words in the sentence, skip the rest of the loop
        #and goes on to next sentence
        if len(word_idx) == 0 : continue
        
        #compute clusters by using a max distance threshold for any two consecutive words
        clusters = []
        cluster = [word_idx[0]]
        
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i-1] < Cluster_threshold:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)
        
        #now score each cluster. The max score for any give cluster is the score for the 
        #sentence
        
        max_cluster_score = 0
        for c in clusters:
            num_important_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * num_important_words_in_cluster\
                    *(num_important_words_in_cluster/total_words_in_cluster)
            
            if score > max_cluster_score:
                max_cluster_score = score 
        score.append(sentence_idx,score)
    return scores
        

In [ ]:
def()

# RATING ALGORITHM

In [65]:
def amazon_rating(x1,x2,x3,x4,x5):
    return (x1+2*x2+3*x3+4*x4+5*x5)/float(x1+x2+x3+x4+x5)

In [66]:
print "50 shades rating", amazon_rating(7708,2732,2970,3584,14876)
print "twightlight rating", amazon_rating(802,349,503,781,4488)
print "the hobbit rating", amazon_rating(141,104,265,1071,6619)
print "daughter of the forest", amazon_rating(20,20,19,73,348)
print "best seller of 2014", amazon_rating(100,108,166,414,1745)
print "divergent", amazon_rating(411,519,1316,3918,14375)
print "eragon", amazon_rating(443,267,331,572,2019)
print "to kill a mocking bird", amazon_rating(141,96,184,657,4708)

50 shades rating 3.47656102918
twightlight rating 4.12725696952
the hobbit rating 4.69792682927
daughter of the forest 4.47708333333
best seller of 2014 4.41966048164
divergent 4.52524465651
eragon 3.95181718062
to kill a mocking bird 4.67559626685


In [67]:
def smooth_rating(x1,x2,x3,x4,x5,alpha,beta):
    return ((x1+2*x2+3*x3+4*x4+5*x5)+alpha*beta)/(float(x1+x2+x3+x4+x5)+beta)

In [68]:
print "50 shades rating", smooth_rating(7708,2732,2970,3584,14876,3,100)
print "twightlight rating", smooth_rating(802,349,503,781,4488,3,100)
print "the hobbit rating", smooth_rating(141,104,265,1071,6619,3,100)
print "daughter of the forest", smooth_rating(20,20,19,73,348,3,100)
print "best seller of 2014", smooth_rating(100,108,166,414,1745,3,100)
print "divergent", smooth_rating(411,519,1316,3918,14375,3,100)
print "eragon", smooth_rating(443,267,331,572,2019,3,100)
print "to kill a mocking bird", smooth_rating(141,96,184,657,4708,3,100)

50 shades rating 3.47507037848
twightlight rating 4.11120603731
the hobbit rating 4.67746987952
daughter of the forest 4.2224137931
best seller of 2014 4.36574249905
divergent 4.51785454722
eragon 3.92631296892
to kill a mocking bird 4.64712878016


In [69]:
def weight_extreme_rating(x1,x2,x3,x4,x5,alpha,beta):
    numer = (2*x1+2*x2+3*x3+4*x4+10*x5)+alpha*beta
    denom = float(2*x1+x2+x3+x4+2*x5+beta)
    return numer/denom

In [70]:
print "50 shades rating", weight_extreme_rating(7708,2732,2970,3584,14876,3,100)
print "twightlight rating", weight_extreme_rating(802,349,503,781,4488,3,100)
print "the hobbit rating", weight_extreme_rating(141,104,265,1071,6619,3,100)
print "daughter of the forest", weight_extreme_rating(20,20,19,73,348,3,100)
print "best seller of 2014", weight_extreme_rating(100,108,166,414,1745,3,100)
print "divergent", weight_extreme_rating(411,519,1316,3918,14375,3,100)
print "eragon", weight_extreme_rating(443,267,331,572,2019,3,100)
print "to kill a mocking bird", weight_extreme_rating(141,96,184,657,4708,3,100)

50 shades rating 3.54118854713
twightlight rating 4.23251847641
the hobbit rating 4.78479415671
daughter of the forest 4.43987341772
best seller of 2014 4.53774006253
divergent 4.67268877911
eragon 4.06700032289
to kill a mocking bird 4.75398230088


In [72]:
def strict_rating(x1,x2,x3,x4,x5,alpha,beta):
    numer = (2*x1+4*x2+3*x3+4*x4+5*x5)+alpha*beta
    denom = float(2*x1+x2+x3+x4+x5+beta)
    return numer/denom

In [79]:
print "50 shades rating", strict_rating(7708,2732,2970,3584,14876,3,100)
print "twightlight rating", strict_rating(802,349,503,781,4488,3,100)
print "the hobbit rating", strict_rating(141,104,265,1071,6619,3,100)
print "daughter of the forest", strict_rating(20,20,19,73,348,3,100)
print "best seller of 2014", strict_rating(100,108,166,414,1745,3,100)
print "divergent", strict_rating(411,519,1316,3918,14375,3,100)
print "eragon", strict_rating(443,267,331,572,2019,3,100)
print "to kill a mocking bird", strict_rating(141,96,184,657,4708,3,100)

50 shades rating 2.7851686152
twightlight rating 3.61365480468
the hobbit rating 4.58086693079
daughter of the forest 4.07903225806
best seller of 2014 4.20437698553
divergent 4.43148035972
eragon 3.48051104374
to kill a mocking bird 4.51151102464


In [88]:
print strict_rating(1,0,3,32,193,2.5,100)
print strict_rating(0,0,1,6,104,2.5,100)
print strict_rating(0,1,7,31,230,2.5,100)
print strict_rating(253,184,297,590,2428,2.5,100)
print strict_rating(101,64,117,205,895,2.5,100)

4.06906906907
3.77725118483
4.19783197832
3.52302631579
3.42258748674


In [87]:
def strict_rating(x1,x2,x3,x4,x5,alpha,beta):
    numer = (3*x1+4*x2+3*x3+4*x4+5*x5)+alpha*beta
    denom = float(5*x1+x2+x3+x4+x5+beta)
    return numer/denom

In [90]:
stopwords = nltk.corpus.stopwords.words('english')

In [93]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

False